In [30]:
import numpy as np
from qiskit import QuantumCircuit
import torch
import matplotlib as plt
import pdflatex
import pylatexenc
from qiskit.circuit import Parameter
from qiskit_machine_learning.neural_networks import SamplerQNN
from qiskit_machine_learning.neural_networks import EstimatorQNN
from qiskit_machine_learning.connectors import TorchConnector
from qiskit.quantum_info import SparsePauliOp
import qiskit_aer
from qiskit_aer.primitives import EstimatorV2
#py -m pip install notebook
#.\venv\Scripts\activate

import time
device = torch.device("cuda")
cpu    = torch.device("cpu")

print("Test")

Test


In [68]:
num_qubits = 8

gpu_estimator = EstimatorV2(options={
            "backend_options": {
            "method": "statevector",
            "device": "GPU"
        }
        })
cpu_estimator = EstimatorV2()


def data_to_encoding_params(input_data):
    #return (input_data / np.max(input_data)) * 2 * np.pi
    return input_data*np.pi

encoding_params = [Parameter(f'theta_{i}') for i in range(num_qubits)]
trainable_params = [Parameter(f'w_{i}') for i in range(num_qubits)]


vqc = QuantumCircuit(num_qubits)

# Encoding
for i, param in enumerate(encoding_params):
    vqc.ry(param, i)

# Hadamard each qubit
for i in range(num_qubits):
    vqc.h(i)

# The Actual VQC
for i, param in enumerate(trainable_params):
    vqc.ry(param, i)


# An observable for each Z-value of each qubit. Effectively telling us what the qubits expectation is in terms of  |0> and |1>
observables = [
SparsePauliOp.from_list([("ZIIIIIII", 1)]),
SparsePauliOp.from_list([("IZIIIIII", 1)]),
SparsePauliOp.from_list([("IIZIIIII", 1)]),
SparsePauliOp.from_list([("IIIZIIII", 1)]),
SparsePauliOp.from_list([("IIIIZIII", 1)]),
SparsePauliOp.from_list([("IIIIIZII", 1)]),
SparsePauliOp.from_list([("IIIIIIZI", 1)]),
SparsePauliOp.from_list([("IIIIIIIZ", 1)])
        ]

qnn_gpu = EstimatorQNN(
        circuit=vqc,
        input_params=encoding_params,
        weight_params=trainable_params,
        observables = observables,
        estimator=gpu_estimator
                )

qnn_cpu = EstimatorQNN(
        circuit=vqc,
        input_params=encoding_params,
        weight_params=trainable_params,
        observables = observables,
                )


input_data = np.array([1,1,1,1, 1, 1, 1, 1])  


encoded_params = data_to_encoding_params(input_data)


weights = np.array([0,0,0,0, 0, 0 ,0 ,0])  

ep_gpu = torch.tensor(encoded_params, device=device)
weights_gpu = torch.tensor([0,0,0,0], device=device)

# Evaluate the QNN

#print("QNN Output:", output)


No gradient function provided, creating a gradient function. If your Estimator requires transpilation, please provide a pass manager.
/tmp/ipykernel_76522/2851644179.py:55: DeprecationWarning: V1 Primitives are deprecated as of qiskit-machine-learning 0.8.0 and will be removed no sooner than 4 months after the release date. Use V2 primitives for continued compatibility and support.
  qnn_cpu = EstimatorQNN(


In [62]:

vqc.draw(output="latex", scale=2)

MissingOptionalLibraryError: "The 'pdflatex' library is required to use 'LaTeX circuit drawing'.  You will likely need to install a full LaTeX distribution for your system."

In [72]:
# Lets experiment with the gpu speedup or naw

t_cpu = []
t_gpu = []

for i in range(10):

    start_time = time.time()
    output = qnn_cpu.forward(encoded_params, weights)
    t_cpu.append(time.time() - start_time)

for i in range(10):

    start_time = time.time()
    output = qnn_gpu.forward(encoded_params, weights)
    t_gpu.append(time.time() - start_time)


print(f"CPU Execution time: {np.mean(t_cpu)}\n")
print(f"GPU Execution time: {np.mean(t_gpu)}\n")


CPU Execution time: 0.0048609495162963865

GPU Execution time: 0.02796337604522705



In [96]:
circuit = QuantumCircuit(num_qubits)
parameters = [Parameter(f'theta_{i}') for i in range(num_qubits)]
input_params = np.random.rand(num_qubits) * np.pi 

# Add some parameterized rotations
for i in range(num_qubits):
    circuit.ry(parameters[i], i)

# Add a Hadamard gate
for i in range(num_qubits):
    circuit.h(i)

# Observables
observable = SparsePauliOp.from_list([("Z" * num_qubits, 1)])

batch_size = 1000
input_params = np.random.rand(batch_size, num_qubits) * np.pi  # Batch of random parameters
batched_inputs = [(circuit, observable, params) for params in input_params]

# Warm up GPU
gpu_estimator.run(batched_inputs[:1])

# Time the CPU-based estimator
start_time = time.time()
cpu_estimator.run(batched_inputs)
cpu_time = time.time() - start_time

# Time the GPU-based estimator
start_time = time.time()
gpu_estimator.run(batched_inputs)
gpu_time = time.time() - start_time

# Print results
#print(f"CPU Estimator Average Execution Time: {np.mean(t_cpu)} seconds")
#print(f"GPU Estimator Average Execution Time: {np.mean(t_gpu)} seconds")

print(f"CPU Estimator Execution Time for Batch: {cpu_time:.5f} seconds")
print(f"GPU Estimator Execution Time for Batch: {gpu_time:.5f} seconds")

CPU Estimator Execution Time for Batch: 1.81887 seconds
GPU Estimator Execution Time for Batch: 0.96174 seconds
